In [2]:
!pip install findspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 6.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 10.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=b04ce80cf439fde31d21a0f6ca76c15f636f33a22e34d6a05bd8153dcf1bae09
  Stored in directory: /home/codespace/.cache/pip/wheels/9c/aa/b1/8433fd8b1afe7eb31196cc74a42cd778bcb52636a428da079d
Successfully built pyspark


In [3]:
import findspark
from pyspark.sql import SparkSession
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [105]:
spark_session = SparkSession.builder.appName('data_film').getOrCreate()

In [199]:
data = spark_session.read.option("delimiter", "\t").csv('data_film.txt', header=False)
data = data.toDF('titre', 'date', 'duree', 'type', 'note', 'nombre avis', 'avis')

# Preprocess

In [200]:
from pyspark.sql import functions as F

In [201]:
data.printSchema()

root
 |-- titre: string (nullable = true)
 |-- date: string (nullable = true)
 |-- duree: string (nullable = true)
 |-- type: string (nullable = true)
 |-- note: string (nullable = true)
 |-- nombre avis: string (nullable = true)
 |-- avis: string (nullable = true)



In [202]:
# variable avis traitement
data = data.withColumn("avis", F.regexp_replace(F.col("avis"), "[\['\"]" , ""))

In [203]:
# traitment de la variable type
data = data.withColumn("type", F.regexp_replace(F.col("type"), "[\[\]',]", ""))


In [204]:
# ne garder que l'heure pour des besoin de modélisation 
data = data.withColumn('duree', F.substring("duree", 1, 2))

In [207]:
# changer le format des dates  
data = data.withColumn("date", F.regexp_replace('date',' ', "-"))
data.show()


+--------------------+-----------------+-----+--------------------+----+-----------+--------------------+
|               titre|             date|duree|                type|note|nombre avis|                avis|
+--------------------+-----------------+-----+--------------------+----+-----------+--------------------+
|   Madeleine Collins| 22-décembre-2021|   1h|               Drame| 3.3|        168|\nMais comment ce...|
|           Les Pires|  7-décembre-2022|   1h|  Comédie dramatique| 3.8|         23|\nSur une idée nu...|
|              Inu-Oh| 23-novembre-2022|   1h|Animation Musical...| 3.6|          8|\nLe scénario est...|
|            Larguées|    18-avril-2018|   1h|             Comédie| 3.3|        364|\n«\xa0Larguées\x...|
|Couleurs de l'inc...|  9-novembre-2022|   2h|    Historique Drame| 4.0|        368|\nLa mort de son ...|
|             Silence|   8-février-2017|   2h|    Drame Historique| 3.6|        383|\nRalalah ! Mais ...|
|Black is Beltza I...| 16-novembre-2022|   1h|

In [206]:
data.show(5)

+--------------------+----------------+-----+--------------------+----+-----------+--------------------+
|               titre|            date|duree|                type|note|nombre avis|                avis|
+--------------------+----------------+-----+--------------------+----+-----------+--------------------+
|   Madeleine Collins|22-décembre-2021|   1h|               Drame| 3.3|        168|\nMais comment ce...|
|           Les Pires| 7-décembre-2022|   1h|  Comédie dramatique| 3.8|         23|\nSur une idée nu...|
|              Inu-Oh|23-novembre-2022|   1h|Animation Musical...| 3.6|          8|\nLe scénario est...|
|            Larguées|   18-avril-2018|   1h|             Comédie| 3.3|        364|\n«\xa0Larguées\x...|
|Couleurs de l'inc...| 9-novembre-2022|   2h|    Historique Drame| 4.0|        368|\nLa mort de son ...|
+--------------------+----------------+-----+--------------------+----+-----------+--------------------+
only showing top 5 rows

